# Tool testing

In [1]:
!pip install -U langchain-community tavily-python python-dotenv langchain-google-genai langgraph langchain-tavily jikanpy-v4

In [2]:
from google.colab import userdata

import os

tkey = userdata.get("TAVILY_API_KEY")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = tkey

gkey = userdata.get("GOOGLE_API_KEY")
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = gkey

In [ ]:
# Import relevant functionality
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [9]:
human_input = input("Enter your question: ")

config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=human_input)]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Enter your question: what is pied piper
================================ Human Message =================================

what is pied piper
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (2003c436-54b3-4f1b-be12-7d706eb312ba)
 Call ID: 2003c436-54b3-4f1b-be12-7d706eb312ba
  Args:
    query: Pied Piper Silicon Valley
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Pied Piper (company) | Silicon Valley Wiki | Fandom", "url": "https://silicon-valley.fandom.com/wiki/Pied_Piper_(company)", "content": "**Pied Piper** was an American tech company based in Silicon Valley, California. The company was formed in 2014 by [Richard Hendricks](/wiki/Richard_Hendricks \"Richard Hendricks\") as a company to develop Richard's [algorithm](/wiki/Pied_Piper_(algorithm) \"Pied Piper (algorithm)\") that he'd created. The company is best known to for achiev

In [10]:
# will invoke the chat protocol, activates user input and outputs the stream of responses.
def talk_tuah():
    hi = input("Enter your question: ")
    config = {"configurable": {"thread_id": "abc123"}}
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content=hi)]},
        config,
        stream_mode="values",
    ):
        if step["messages"][-1].type == "ai":
            print(step["messages"][-1].content)

In [11]:
talk_tuah()

Enter your question: what is One Piece

One Piece is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he strives to become the Pirate King and find the legendary treasure known as "One Piece". It is the best-selling manga series in history.


Basic implementation of agent with memory works, displayin only ai message aka the response works too, next step is to develop a tool for every function in the jikan api.